## Решение практической задачи 2

### Задача: прогнозирование потребительских предпочтений на промышленные товары

<hr>

In [1]:
import os  # Взаимодействие с файловой системой
import sys # Доступ к некоторым переменным и функциям Python

PATH_TO_SOURCE = os.path.abspath(os.path.dirname(globals()['_dh'][0]))
PATH_TO_ROOT = os.path.join(PATH_TO_SOURCE, '..', '..', '..')

sys.path.insert(0, os.path.abspath(PATH_TO_ROOT))

In [2]:
# Импорт необходимых инструментов
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio']['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url)

url = _b5.weights_for_big5_['audio']['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc()
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video']['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url)

url = _b5.weights_for_big5_['video']['fe']['sberdisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url)

url = _b5.weights_for_big5_['video']['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url)

# Формирование модели для мультимодального объединения информации
res_load_model_b5 = _b5.load_av_models_b5()

# Загрузка весов модели для мультимодального объединения информации
url_openness = _b5.weights_for_big5_['av']['b5']['openness']['sberdisk']
url_conscientiousness = _b5.weights_for_big5_['av']['b5']['conscientiousness']['sberdisk']
url_extraversion = _b5.weights_for_big5_['av']['b5']['extraversion']['sberdisk']
url_agreeableness = _b5.weights_for_big5_['av']['b5']['agreeableness']['sberdisk']
url_neuroticism = _b5.weights_for_big5_['av']['b5']['neuroticism']['sberdisk']

res_load_models_weights_b5 = _b5.load_av_models_weights_b5(
    url_openness = url_openness, # Открытость опыту
    url_conscientiousness = url_conscientiousness, # Добросовестность
    url_extraversion = url_extraversion, # Экстраверсия
    url_agreeableness = url_agreeableness, # Доброжелательность
    url_neuroticism = url_neuroticism, # Нейротизм
)

PATH_TO_DIR = './video_FI/'
PATH_SAVE_VIDEO = './video_FI/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса First Impression V2
# URL: https://chalearnlap.cvc.uab.cat/dataset/24/description/
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '429713680?token=FqHdMLSSh7zYSZt&filename=_plk5k7PBEg.003.mp4',
    '429713681?token=Hz9b4lQkrLfic33&filename=be0DQawtVkE.002.mp4',
    '429713683?token=EgUXS9Xs8xHm5gz&filename=2d6btbaNdfo.000.mp4',
    '429713684?token=1U26753kmPYdIgt&filename=300gK3CnzW0.003.mp4',
    '429713685?token=LyigAWLTzDNwKJO&filename=300gK3CnzW0.001.mp4',
    '429713686?token=EpfRbCKHyuc4HPu&filename=cLaZxEf1nE4.004.mp4',
    '429713687?token=FNTkwqBr4jOS95l&filename=g24JGYuT74A.004.mp4',
    '429713688?token=qDT95nz7hfm2Nki&filename=JZNMxa3OKHY.000.mp4',
    '429713689?token=noLguEGXDpbcKhg&filename=nvlqJbHk_Lc.003.mp4',
    '429713679?token=9L7RQ0hgdJlcek6&filename=4vdJGgZpj4k.003.mp4'
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = False)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mp4'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_['sberdisk']

_b5.get_av_union_predictions(url_accuracy = url_accuracy)

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-08 23:41:40</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... Users/dl/GitHub/OCEANAI/docs/source/user_guide/notebooks/video_FI/test/be0DQawtVkE.002.mp4 ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism
ID,,,,,,
1,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.57965,0.607485,0.50449,0.592074,0.551907
2,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.657927,0.579338,0.605996,0.59566,0.614946
3,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.555637,0.478637,0.4257,0.507637,0.442785
4,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.59052,0.632427,0.508794,0.643603,0.593294
5,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.454759,0.441814,0.375909,0.515255,0.427
6,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.547811,0.355932,0.413966,0.510216,0.439228
7,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.643725,0.583322,0.615071,0.592651,0.618553
8,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.479282,0.45275,0.383508,0.53317,0.480088
9,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.472738,0.6322,0.351847,0.579884,0.514094


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-08 23:41:40</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,Mean
Metrics,,,,,,
MAE,0.0636,0.0497,0.0908,0.0751,0.0612,0.0681
Accuracy,0.9364,0.9503,0.9092,0.9249,0.9388,0.9319


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-08 23:41:40</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.0681</span>, средняя точность: <span style="color:#1776D2">0.9319</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 149.127 сек. ---**</span>

True

<hr>

Для прогнозирование потребительских предпочтений на промышленные товары неоходимо знать коэффициенты корреляции, определяющие взаимосвязь между персональными качествами личности человека и предпочтениями в товарах или услугах.

В качестве примера предлагается использование коэффициентов корреляции между персональными качествами человека и характеристиками автомобилей, представленных в статье:

1) O'Connor P. J. et al. What Drives Consumer Automobile Choice? Investigating Personality Trait Predictors of Vehicle Preference Factors // Personality and Individual Differences. – 2022. – Vol. 184. – pp. 111220.

Пользователь может установит свои коэффициенты корреляции.

In [4]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/477865164?token=Ai6WOMPVZ0C3f6W&filename=auto_characteristics.csv'
df_correlation_coefficients = pd.read_csv(url)
df_correlation_coefficients = pd.DataFrame(
    df_correlation_coefficients.drop(['Style and performance', 'Safety and practicality'], axis = 1)
)
df_correlation_coefficients.index.name = 'ID'
df_correlation_coefficients.index += 1
df_correlation_coefficients.index = df_correlation_coefficients.index.map(str)

df_correlation_coefficients

,Trait,Performance,Classic car features,Luxury additions,Fashion and attention,Recreation,Technology,Family friendly,Safe and reliable,Practical and easy to use,Economical/low cost,Basic features
ID,,,,,,,,,,,,
1,Openness,0.020000,-0.033333,-0.030000,-0.050000,0.033333,0.013333,-0.030000,0.136667,0.106667,0.093333,0.006667
2,Conscientiousness,0.013333,-0.193333,-0.063333,-0.096667,-0.096667,0.086667,-0.063333,0.280000,0.180000,0.130000,0.143333
3,Extraversion,0.133333,0.060000,0.106667,0.123333,0.126667,0.120000,0.090000,0.136667,0.043333,0.073333,0.050000
4,Agreeableness,-0.036667,-0.193333,-0.133333,-0.133333,-0.090000,0.046667,-0.016667,0.240000,0.160000,0.120000,0.083333
5,Neuroticism,0.016667,-0.006667,-0.010000,-0.006667,-0.033333,0.046667,-0.023333,0.093333,0.046667,0.046667,-0.040000


In [5]:
_b5._priority_calculation(
    correlation_coefficients = df_correlation_coefficients,
    col_name_ocean = 'Trait',
    threshold = 0.55,
    number_priority = 3,
    number_importance_traits = 3,
    out = False
)

_b5._save_logs(df = _b5.df_files_priority_, name = 'auto_characteristics_priority', out = True)

_b5.df_files_priority_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,Priority 1,Priority 2,Priority 3,Trait importance 1,Trait importance 2,Trait importance 3
ID,,,,,,,,,,,,
1,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.57965,0.607485,0.50449,0.592074,0.551907,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness
2,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.657927,0.579338,0.605996,0.59566,0.614946,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness
3,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.555637,0.478637,0.4257,0.507637,0.442785,Classic car features,Recreation,Luxury additions,Agreeableness,Conscientiousness,Openness
4,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.59052,0.632427,0.508794,0.643603,0.593294,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness
5,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.454759,0.441814,0.375909,0.515255,0.427,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness
6,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.547811,0.355932,0.413966,0.510216,0.439228,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness
7,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.643725,0.583322,0.615071,0.592651,0.618553,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness
8,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.479282,0.45275,0.383508,0.53317,0.480088,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness
9,/Users/dl/GitHub/OCEANAI/docs/source/user_guid...,0.472738,0.6322,0.351847,0.579884,0.514094,Safe and reliable,Basic features,Practical and easy to use,Conscientiousness,Agreeableness,Neuroticism
